In [2]:
# Importing

# modules
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import datetime

# functions
from implementations import *
from helpers import *

#data
(labels, data, ids) = load_csv_data("data/train.csv")  # load data


OSError: data/train.csv not found.